# INSERT

In [1]:
import pymysql
import pandas as pd
import numpy as np
from config import *
import uuid
import datetime 
from itertools import product

In [2]:
def query(sql, data):
    """
    DB에 SQL문 실행(query: 데이터베이스에 정보 요청)
       Args:
           sql(str): sql 구문
           data(list or tuple): sql 파라미터. ex) [val1, val2 ...]
       Returns:
           query 결과. 결과가 없으면 빈 tuple 리턴
    """
    # connect(): DB와 연결
    db = pymysql.Connect(host=MYSQL_HOST, 
                     user=MYSQL_USER, 
                     password=MYSQL_PASSWORD, 
                     database=MYSQL_DATABASE,
                     autocommit=MYSQL_AUTOCOMMIT, # Query 실행 후 자동 commit(실제 insert, update 반영)
                     charset=MYSQL_CHARSET, # 인코딩 정보
                     cursorclass=pymysql.cursors.DictCursor # DB를 조회한 결과를 Column 명이 Key인 Dictionary로 저장
                    )
    cursor = db.cursor()

    cursor.execute(sql, data) 
    result = cursor.fetchall() # fetchall(): 모든 데이터를 한 번에 가져옴
    db.close()
    return to_pandas(result)

def to_pandas(result):
    return pd.DataFrame(result)

In [3]:
# 테이블의 전체 데이터를 select
sql = "SELECT * FROM " + MYSQL_TABLENAME+ ";"
print(sql)
select_all = query(sql, ())
select_all.head()

SELECT * FROM mytable;


,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
0,1,b177b428b46311edbb8940b076a1ec6b,2023-02-24 16:52:59,Choi,Choi@mysql.com,010-9876-5432,35,"[{""search"": ""naver""}, {""viewing time"": ""30""}]",,
1,2,b1778d08b46311edae6040b076a1ec6b,2023-02-24 16:52:59,Lee,Lee@mysql.com,010-4567-8910,80,"[{""search"": ""youtube""}, {""viewing time"": ""54""}]",,
2,3,b1773ed9b46311edaa0240b076a1ec6b,2023-02-24 16:52:59,Kim,Kim@mysql.com,010-1234-5678,44,"[{""search"": ""github""}, {""viewing time"": ""64""}]",,
3,4,b177b429b46311edbf3540b076a1ec6b,2023-02-24 15:52:59,Choi,Choi@mysql.com,010-9876-5432,29,"[{""search"": ""github""}, {""viewing time"": ""33""}]",,
4,5,b1778d09b46311ed8d5f40b076a1ec6b,2023-02-24 15:52:59,Lee,Lee@mysql.com,010-4567-8910,0,"[{""search"": ""naver""}, {""viewing time"": ""19""}]",,


In [4]:
# 테이블의 전체 데이터를 select
sql = "SELECT * FROM " + MYSQL_TABLENAME+ ";"
print(sql)
result = query(sql, ())
result.head()

SELECT * FROM mytable;


,index,uuid,reg_date,name,email,phone,energy,label,mark,ext01
0,1,b177b428b46311edbb8940b076a1ec6b,2023-02-24 16:52:59,Choi,Choi@mysql.com,010-9876-5432,35,"[{""search"": ""naver""}, {""viewing time"": ""30""}]",,
1,2,b1778d08b46311edae6040b076a1ec6b,2023-02-24 16:52:59,Lee,Lee@mysql.com,010-4567-8910,80,"[{""search"": ""youtube""}, {""viewing time"": ""54""}]",,
2,3,b1773ed9b46311edaa0240b076a1ec6b,2023-02-24 16:52:59,Kim,Kim@mysql.com,010-1234-5678,44,"[{""search"": ""github""}, {""viewing time"": ""64""}]",,
3,4,b177b429b46311edbf3540b076a1ec6b,2023-02-24 15:52:59,Choi,Choi@mysql.com,010-9876-5432,29,"[{""search"": ""github""}, {""viewing time"": ""33""}]",,
4,5,b1778d09b46311ed8d5f40b076a1ec6b,2023-02-24 15:52:59,Lee,Lee@mysql.com,010-4567-8910,0,"[{""search"": ""naver""}, {""viewing time"": ""19""}]",,


## INSERT IGNORE
- 중복 키 에러가 발생했을 때 신규로 입력되는 레코드를 무시하는 방법(UNIQUE KEY가 지정되어 있어야함)

In [5]:
uuid_hex = uuid.uuid4().hex
now = datetime.datetime.utcnow().timestamp()# utc로 현재 시간 얻고 계산
reg_date = str(datetime.datetime.fromtimestamp(now))

In [6]:
result = [
        uuid_hex,
        reg_date,
        "test",
        "test@mytable.com",
        "123-4567-8910",
        100,
        """[{"search": "naver"}, {"viewing time": "19"}]""",
        "",
        ""
    ]

In [7]:
data = result

In [8]:
data

['bba9523e29854bb4a4ad05315dd997e4',
 '2023-02-25 19:57:00.856096',
 'test',
 'test@mytable.com',
 '123-4567-8910',
 100,
 '[{"search": "naver"}, {"viewing time": "19"}]',
 '',
 '']

In [9]:
sql = "INSERT IGNORE INTO " + MYSQL_TABLENAME + """ (uuid, reg_date, name, email, phone, energy, label, mark, ext01) 
VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);"""
result = query(sql, data)
print(result)

Empty DataFrame
Columns: []
Index: []


## INSERT + WHERE NOT EXISTS
- NOT EXISTS: 조건을 만족하는 데이터가 있는지 확인. 조건 만족 시, FLASE, 아니면 TRUE 
- 조건문을 활용한 INSERT

In [10]:
sql = """SELECT "3","4" WHERE TRUE;"""
result = query(sql, ())
print(result)

   3  4
0  3  4


In [11]:
sql = """SELECT "3","4" WHERE FALSE;"""
result = query(sql, ())
print(result)

Empty DataFrame
Columns: []
Index: []


In [12]:
sql = "INSERT INTO " + MYSQL_TABLENAME + """ (uuid, reg_date, name, email, phone, energy, label, mark, ext01) 
SELECT %s,%s,%s,%s,%s,%s,%s,%s,%s WHERE NOT EXISTS (SELECT * FROM """\
+MYSQL_TABLENAME+\
""" WHERE name = "test" and email="test@mytable.com");"""
result = query(sql, data)
print(result)

Empty DataFrame
Columns: []
Index: []
